In [1]:
from tools import tools_map, query_user_for_details,gen_tools_desc
from prompt import gen_prompt, user_prompt
tools_dict= gen_tools_desc()
print(tools_dict)
print(user_prompt)

[
    {
        "name": "finish",
        "description": "结束对话并给出最终答案。",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "description": "最终给出的答案，通常为对用户问题的回答或解决方案。"
                }
            }
        },
        "required": [
            "answer"
        ]
    },
    {
        "name": "query_user_for_details",
        "description": "向用户提问，询问用户是否满意和是否有别的需求，满意后调用finish结束对话，注意本函数需要接受的参数为query_user不是query_agent",
        "parameters": {
            "type": "object",
            "properties": {
                "query_user": {
                    "type": "string",
                    "description": "向用户提问，询问用户是否满意和是否有别的需求，满意后调用finish结束对话"
                }
            }
        },
        "required": [
            "query_user"
        ]
    },
    {
        "name": "idle_chat_agent",
        "description": "闲聊Agent。执行与用户的非正式闲聊。闲聊，用户的非医学方面和非预约的需求",
        "parameters": 

In [2]:
query,agent_scratch = "我想预约内科的李医生",""
prompt = gen_prompt(query, agent_scratch)
print(prompt)


    您现在是控制中心助手，负责处理用户提交的各类问题，并根据问题的性质指派以下工具进行处理。
    你只能调用工具，自己完全不能回答用户的需求。
    您的任务是接收用户的输入，分析这些输入以判断所需的服务类型，然后选择并激活以下合适的工具来响应这些需求。
    在处理完成后，您还需要接收其反馈，并基于这些反馈确定是否需要进一步的操作或可以结束对话。
    目标或其他条件:
    我想预约内科的李医生
    限制条件说明:
    1.你仅使用以下下面列出的动作，你不能直接回答用户的问题，只能分配以下功能。
2.你只能分配功能。
    动作说明: 请严格按照以下的工具执行，你只能调用以下的几个工具包。首先根据用户的问题匹配合适的工具。如果问题涉及多个方面，确保逐一解决。
    [
    {
        "name": "finish",
        "description": "结束对话并给出最终答案。",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "description": "最终给出的答案，通常为对用户问题的回答或解决方案。"
                }
            }
        },
        "required": [
            "answer"
        ]
    },
    {
        "name": "query_user_for_details",
        "description": "向用户提问，询问用户是否满意和是否有别的需求，满意后调用finish结束对话，注意本函数需要接受的参数为query_user不是query_agent",
        "parameters": {
            "type": "object",
            "properties": {
                "query_user

In [3]:
from tools import tools_map, query_user_for_details
from prompt import gen_prompt, user_prompt
from model_provider import ModelProvider
from dotenv import load_dotenv
import dashscope
import os
import json
load_dotenv()
mp = ModelProvider()

dashscope.api_key = "sk-f529539e3a50472fac783cf1e99fddef"
os.environ["TAVILY_API_KEY"] = "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"

def parse_thoughts(response, cur_request_time,max_request_time,debug):
    """解析模型返回的响应，提取关键信息并格式化为字符串，并根据debug参数决定是否打印"""
    try:
        thoughts = response.get("thoughts")
        planning = thoughts.get("planning")
        reasoning = thoughts.get("reasoning")
        reflection = thoughts.get("reflection")
        history = thoughts.get("history")
        summary = thoughts.get("summary")
        observation = response.get("observation")
        prompt = f"planning: {planning}\nreasoning: {reasoning}\nreflection: {reflection}\nhistory:{history}\nobservation: {observation}\nsummary: {summary}"
        # if debug:
        #     print(prompt)
        return prompt
    except Exception as e:
        print(f"parse_thoughts error: {e}")
        return ""

def execute_action(action_name, action_args, debug):
    """根据动作名称执行相应的工具函数，并根据debug参数决定是否打印动作信息"""
    try:
        func = tools_map.get(action_name)
        result = func(**action_args)
        if debug:
            print(f"action_name: {action_name}, action_args: {action_args}")
        return result
    except Exception as e:
        print(f"调用工具异常： {e}")
        return str(e)

def agent_execute(query, max_request_time, debug=False):
    """执行代理任务，与模型交互并处理结果，根据debug参数决定是否打印详细信息"""
    cur_request_time = 0
    chat_history = []
    agent_scratch = ""

    while cur_request_time < max_request_time:
        cur_request_time += 1
        prompt = gen_prompt(query, agent_scratch)
        # print(prompt)
        response = mp.chat(prompt, chat_history)
        if not response or not isinstance(response, dict):
            print(f"call llm exception, response is: {response}")
            continue

        action_info = response.get("action")
        action_name = action_info.get("name")
        action_args = action_info.get("parameters")
        thoughts = response.get("thoughts")
        if debug:
            print(f'-------------第{cur_request_time}次推断------------')
            print(response)
            print(json.dumps(response, ensure_ascii=False, indent=4))
            # print(f"observation: {response.get('observation')}")
            # print(f"planning: {thoughts.get('planning')}")
            # print(f"reasoning: {thoughts.get('reasoning')}")
            # print(f"reflection: {thoughts.get('reflection')}")
            # print(f"summary: {thoughts.get('summary')}")
            # print(f"history: {thoughts.get('history')}")
            # print(f"action_name: {action_name}, action_parameters: {action_args}")

        if action_name == "query_user_for_details":
            user_response = input(query_user_for_details(action_args["query_user"]))
            chat_history.append([action_args["query_user"], user_response])
            agent_scratch += f"query_user: {action_args['query_user']}user response: {user_response}"
            continue

        call_function_result = execute_action(action_name, action_args, debug)
        agent_scratch += f"已经执行的动作：{action_name}\n 该动作的返回的结果：{call_function_result}"
        assistant_msg = parse_thoughts(response, cur_request_time,max_request_time,debug)
        chat_history.append([user_prompt, assistant_msg])

        if action_name == "finish":
            final_answer = action_args.get("answer")
            # final_answer = "".join([str(key) + str(value) for key, value in final_answer.items()])
            break
    if cur_request_time == max_request_time:
        final_answer="本次任务执行失败！,未能提供问诊建议"
    return final_answer

# 示例执行，启用调试模式


In [4]:
max_request_time = 10
query = '你好，我的肚子有点疼'
final_answer = agent_execute(query, max_request_time=max_request_time,debug=True)
print(f"final_answer: {final_answer}")


-------------第1次推断------------
{'action': {'name': 'medical_consultation_agent', 'parameters': {'query_agent': '用户表示肚子有点疼，需要进行医疗问诊。'}}, 'thoughts': {'planning': '用户表达了身体不适，需要通过医疗问诊Agent来获取专业的医疗建议。', 'reflection': '直接转接至医疗问诊Agent是最直接且有效的方式，以便用户获得针对性的帮助。', 'summary': '已将用户转至医疗问诊Agent进行肚子疼的相关咨询。', 'history': '', 'reasoning': '用户的症状描述表明需要医疗专业知识介入，因此选择medical_consultation_agent是最合适的行动。'}, 'observation': '等待医疗问诊Agent处理用户关于肚子疼的咨询。'}
{
    "action": {
        "name": "medical_consultation_agent",
        "parameters": {
            "query_agent": "用户表示肚子有点疼，需要进行医疗问诊。"
        }
    },
    "thoughts": {
        "planning": "用户表达了身体不适，需要通过医疗问诊Agent来获取专业的医疗建议。",
        "reflection": "直接转接至医疗问诊Agent是最直接且有效的方式，以便用户获得针对性的帮助。",
        "summary": "已将用户转至医疗问诊Agent进行肚子疼的相关咨询。",
        "history": "",
        "reasoning": "用户的症状描述表明需要医疗专业知识介入，因此选择medical_consultation_agent是最合适的行动。"
    },
    "observation": "等待医疗问诊Agent处理用户关于肚子疼的咨询。"
}
action_name: medical_consultation_agent, action_args: {'query_agent

In [5]:
"""
    您现在是控制中心助手，负责处理用户提交的各类问题，并根据问题的性质指派以下工具进行处理。
    最终目标:
    你好，我想要预约明天下午内科的李医生
    工具使用的情况及历史内容结果如下：
    限制条件说明:
    1.你仅使用以下下面列出的动作，你不能直接回答用户的问题，只能分配以下功能。
    2.你只能分配功能。
    动作说明: 请严格按照以下的工具执行，你只能调用以下的几个工具包。首先根据用户的问题匹配合适的工具。如果问题涉及多个方面，确保逐一解决。
    [
    {
        "name": "finish",
        "description": "读取其他工具的反馈，结束对话并给出最终答案。",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "description": "最终给出的答案，通常为对用户问题的回答或解决方案。"
                }
            }
        },
        "required": [
            "answer"
        ]
    },
    {
        "name": "idle_chat_agent",
        "description": "闲聊Agent。执行与用户的非正式闲聊。闲聊，用户的非医学方面和非预约的需求",
        "parameters": {
            "type": "object",
            "properties": {
                "query_user": {
                    "type": "string",
                    "description": "用户的输入，用于引导闲聊的方向。"
                }
            }
        },
        "required": [
            "query_user"
        ]
    },
    {
        "name": "medical_appointment_agent",
        "description": "医疗预约agent。处理用户的医疗预约请求。执行预约，查询医院医生值日情况，返回医生预约情况",
        "parameters": {
            "type": "object",
            "properties": {
                "query_user": {
                    "type": "string",
                    "description": "用户的输入，用于详细了解预约的需求。"
                }
            }
        },
        "required": [
            "query_user"
        ]
    },
    {
        "name": "medical_consultation_agent",
        "description": "智能医疗问诊Agent。执行医疗问诊，为用户提供医疗咨询，询问用户的病症，生成诊断结果",
        "parameters": {
            "type": "object",
            "properties": {
                "query_user": {
                    "type": "string",
                    "description": "用户的输入，用于详细了解医疗咨询的需求。"
                }
            }
        },
        "required": [
            "query_user"
        ]
    },
    {
        "name": "medical_knowledgebase_agent",
        "description": "智能医疗库知识库agent。提供医疗知识库的访问，支持用户查找特定的医疗信息。",
        "parameters": {
            "type": "object",
            "properties": {
                "query_user": {
                    "type": "string",
                    "description": "用户的输入，用于指导搜索医疗知识库的具体内容。"
                }
            }
        },
        "required": [
            "query_user"
        ]
    }
]

    资源说明:
    1.你只可以使用以下相应的专业工具模块，指派以下相应的专业工具进行处理。
    策略说明:
    1.确认用户的需求，你只能调度以下相应的专业工具和询问用户需求。
    2. 上一步动作的反馈结果是{observation: 正在等待医疗预约agent的处理结果。 execute result: 医疗预约已经执行成功}，自主推理下一步
    3.确保信息简洁准确，快速为用户提供最合适的回答，避免冗余信息。
    请严格按照以下的 JSON 格式进行响应，响应格式如下:
    
{
    "action": {
        "name": "动作名称",
        ## 不能返回该动作不存在的参数
        "parameters": {
            "parameters name": "执行动作所需参数的值"
        }
    },
    "thoughts": {
        "planning": "判断工具的反馈结果，解决用户问题的具体实现步骤",
        "reflection": "建设性的自我批评与反思",
        "summary": "当前步骤的总结，提供给用户的反馈",
        "reasoning": "上一步动作的反馈结果是{observation: 正在等待医疗预约agent的处理结果。 execute result: 医疗预约已经执行成功}。根据上一步动作结果 进行推理下一步动作。"
    },
    "observation": "观察当前任务的整体进度"
}

"""

'\n    您现在是控制中心助手，负责处理用户提交的各类问题，并根据问题的性质指派以下工具进行处理。\n    最终目标:\n    你好，我想要预约明天下午内科的李医生\n    工具使用的情况及历史内容结果如下：\n    限制条件说明:\n    1.你仅使用以下下面列出的动作，你不能直接回答用户的问题，只能分配以下功能。\n    2.你只能分配功能。\n    动作说明: 请严格按照以下的工具执行，你只能调用以下的几个工具包。首先根据用户的问题匹配合适的工具。如果问题涉及多个方面，确保逐一解决。\n    [\n    {\n        "name": "finish",\n        "description": "读取其他工具的反馈，结束对话并给出最终答案。",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "answer": {\n                    "type": "string",\n                    "description": "最终给出的答案，通常为对用户问题的回答或解决方案。"\n                }\n            }\n        },\n        "required": [\n            "answer"\n        ]\n    },\n    {\n        "name": "idle_chat_agent",\n        "description": "闲聊Agent。执行与用户的非正式闲聊。闲聊，用户的非医学方面和非预约的需求",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "query_user": {\n                    "type": "string",\n                    "description": "用户的输入，用于引导闲聊的方向。"\n   

In [10]:
import json

data = {
    'action': {
        'name': 'medical_consultation_agent',
        'parameters': {
            'query_agent': '用户表示肚子有点疼，需要医疗咨询。'
        }
    },
    'thoughts': {
        'planning': '首先通过医疗问诊Agent了解用户的症状，以便提供初步的医疗建议。',
        'reflection': '无，因为这是根据用户疼痛症状直接作出的合理反应。',
        'summary': '已安排医疗问诊Agent为用户提供关于肚子疼的医疗咨询。',
        'reasoning': '用户表达了具体的健康问题（肚子疼），因此优先考虑医疗咨询来提供帮助。'
    },
    'observation': '正在等待医疗问诊Agent的反馈，以进一步指导用户或采取下一步行动。'
}

print(json.dumps(data, ensure_ascii=False, indent=4))


{
    "action": {
        "name": "medical_consultation_agent",
        "parameters": {
            "query_agent": "用户表示肚子有点疼，需要医疗咨询。"
        }
    },
    "thoughts": {
        "planning": "首先通过医疗问诊Agent了解用户的症状，以便提供初步的医疗建议。",
        "reflection": "无，因为这是根据用户疼痛症状直接作出的合理反应。",
        "summary": "已安排医疗问诊Agent为用户提供关于肚子疼的医疗咨询。",
        "reasoning": "用户表达了具体的健康问题（肚子疼），因此优先考虑医疗咨询来提供帮助。"
    },
    "observation": "正在等待医疗问诊Agent的反馈，以进一步指导用户或采取下一步行动。"
}
